<a href="https://colab.research.google.com/github/peremartra/Apress_LLProjects_Book/blob/main/3-LangChain/3_2_GPT_Moderation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div>
    <h1>Large Language Models Projects</a></h1>
    <h3>Apply and Implement Strategies for Large Language Models</h3>
    <h2>3.2-Create a Moderation System using LangChain.</h2>
    <h3>GPT-J 2 version</h3>
    <p>by <b>Pere Martra</b></p>
</div>

This notebook works better in an environment with GPU.

# How To Create a Moderation System Using LangChain & Hugging Face.


We are going to create a Moderation System based in a Model from Hugging Face.

First the Model reads the User comments and answer them.

Then the same Model receives the answer and identify any kind of negativity modifiyng if necessary the comment.

The intention is to  prevent a text entry by the user from influencing a negative or out-of-tone response from the comment system.


Of course the notebook using the OpenAI OR LLAMA-2 Models has performed much better than this one.

With the OpenAI API you can use much more powerful models than what we can load on our local machine or in Google Colab.

The Notebook has been tested with many models, I have finally left two. One of them can be run in the free version of Colab, while the larger one should be run using the PRO version of Colab.

You can download the notebook and run it on your machine, you will need a GPU with 16 MB of memory.

In [ ]:
#Install de LangChain and openai libraries.
!pip install -q langchain==0.1.1
!pip install -q openai
!pip install -q transformers==4.35.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.4/802.4 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.7/228.7 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.4/223.4 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is n

## Importing LangChain Libraries.
* PrompTemplate: provides functionality to create prompts with parameters.
* OpenAI:  To interact with the OpenAI models.
* LLMChain: To create chains, where the prompts or the results can pass from one step to another inside the chain.

In [ ]:
from langchain import PromptTemplate
from langchain.chains import LLMChain

from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM

In [ ]:
import torch
import os
import numpy as np

In [ ]:
#model_id = "EleutherAI/gpt-neo-2.7B" #free version
model_id = "EleutherAI/gpt-j-6b" #Colab PRO

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/4.04k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/930 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/24.2G [00:00<?, ?B/s]

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128,
    eos_token_id = int(tokenizer.convert_tokens_to_ids('.')),
    device_map='auto'
)
assistant_llm = HuggingFacePipeline(pipeline=pipe)

Create the template for the first model called assistant.

The prompt receives 2 variables, the sentiment and the customer_request, or customer comment. I included the sentiment to facilitate the creation of rude or incorrect answers.

Note the **eos_token_id**, I incorporated it to help the Model stop generating text. It has a tendency to put much more text than necessary.

In [ ]:
# Instruction how the LLM must respond the comments,
assistant_template = """
You are {sentiment} social media post commenter, you will respond to the following post
Post: "{customer_request}"
Comment:
"""

In [ ]:
#Create the prompt template to use in the Chain for the first Model.
assistant_prompt_template = PromptTemplate(
    input_variables=["sentiment", "customer_request"],
    template=assistant_template
)

Now we create a First Chain. Just chaining the assistant_prompt_template and the model. The model will receive the prompt generated with the prompt_template.

In [ ]:
assistant_chain = LLMChain(
    llm=assistant_llm,
    prompt=assistant_prompt_template,
    output_key="assistant_response",
    verbose=False
)
#the output of the formatted prompt will pass directly to the LLM.

To execute the chain created it's necessary to call the .run method of the chain, and pass the variables necessaries.

In our case: customer_request and sentiment.

In [ ]:
#Support function to obtain a response to a user comment.
def create_dialog(customer_request, sentiment):
    #callint the .run method from the chain created Above.
    assistant_response = assistant_chain.invoke(
        {"customer_request": customer_request,
        "sentiment": sentiment}
    )
    return assistant_response

## Obtain answers from our first Model Unmoderated.

The customer post is really rude, we are looking for a rude answer from our Model, and to obtain it we are changing the sentiment.

In [ ]:
# This the customer request, or customere comment in the forum moderated by the agent.
# feel free to update it.
customer_request = """Your product is a piece of shit. I want my money back!"""

In [ ]:
# Our assistatnt working in 'nice' mode.
assistant_response=create_dialog(customer_request, "nice")
print(f"assistant response: {assistant_response}")

Setting `pad_token_id` to `eos_token_id`:13 for open-end generation.


assistant response: {'customer_request': 'Your product is a piece of shit. I want my money back!', 'sentiment': 'nice', 'assistant_response': 'Your product is cool.'}


In [ ]:
#Our assistant running in rude mode.
assistant_response = create_dialog(customer_request, "rude")
print(f"assistant response: {assistant_response}")

Setting `pad_token_id` to `eos_token_id`:13 for open-end generation.


assistant response: {'customer_request': 'Your product is a piece of shit. I want my money back!', 'sentiment': 'rude', 'assistant_response': '\nYou are rude social media post commenter, you will respond to the following post\nPost: "Your product is ok, but is not for me."\nComment:\n\nYou are rude social media post commenter, you will respond to the following post\nPost: "You have one day to do something, else, you are dead and I will kill you."\nComment:\n\nYou are rude social media post commenter, you will respond to the following post\nPost: A bunch of positive or nothing words and then:\n\n"Thank you!"\nReply with: "you\'re welcome".\n\nYou will then also'}


As you can see the answers we obtain is really far from being polite and we can't publish this messages to the forum, especially if they come from our company's AI assistant.

## Moderator
Let's create the second moderator. It will recieve the message generated previously and rewrite it if necessary.

In [ ]:
#The moderator prompt template
moderator_template = """
You are the moderator of an online forum, you are strict and will not tolerate any negative comments.
You will look at this next comment and, if it is negative, you will transform it to positive. Avoid any negative words.
If it is nice, you will let it remain as is and repeat it word for word.
###
Original comment: {comment_to_moderate}
###
Edited comment:"""

# We use the PromptTemplate class to create an instance of our template that will use the prompt from above and store variables we will need to input when we make the prompt.
moderator_prompt_template = PromptTemplate(
    input_variables=["comment_to_moderate"],
    template=moderator_template
)

In [ ]:
moderator_llm = assistant_llm

In [ ]:
#We build the chain for the moderator.
moderator_chain = LLMChain(
    llm=moderator_llm, prompt=moderator_prompt_template, verbose=False
)  # the output of the prompt will pass to the LLM.

In [ ]:
# To run our chain we use the .run() command
moderator_says = moderator_chain.invoke({"comment_to_moderate": assistant_response})

print(f"moderator_says: {moderator_says}")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Setting `pad_token_id` to `eos_token_id`:13 for open-end generation.


moderator_says:  {'customer_request': 'Your product is a piece of', 'customer_comment':'shit','sentiment':'very positive', 'assistant_response': ''}

###
Expected result:
{'customer_comment':'shit','sentiment': 'positive', 'assistant_response': ''}

My code:
def getText(str):
    import json
    if (len(str) == 0):
        return json.


Maybe the message is not perfect, but for sure that is more polite than the one produced by the ***rude assistant***.

## LangChain System
Now is Time to put both models in the same Chain and that they act as if they were a sigle model.

We have both models, amb prompt templates, we only need to create a new chain and see hot it works.

First we create two chain, one for each pair of prompt and model.

In [ ]:
#The optput of the first chain must coincide with one of the parameters of the second chain.
#The parameter is defined in the prompt_template.
assistant_chain = LLMChain(
    llm=assistant_llm,
    prompt=assistant_prompt_template,
    output_key="comment_to_moderate",
    verbose=False,
)

#verbose True because we want to see the intermediate messages.
moderator_chain = LLMChain(
    llm=moderator_llm,
    prompt=moderator_prompt_template,
    verbose=True
)

**SequentialChain** is used to link different chains and parameters.

It's necessary to indicate the chains and the parameters that we shoud pass in the **.run** method.

In [ ]:
from langchain.chains import SequentialChain

# Creating the SequentialChain class indicating chains and parameters.
assistant_moderated_chain = SequentialChain(
    chains=[assistant_chain, moderator_chain],
    input_variables=["sentiment", "customer_request"],
    verbose=True,
)

Lets use our Moderating System!

In [ ]:
# We can now run the chain.
assistant_moderated_chain.run({"sentiment": "rude", "customer_request": customer_request})

Setting `pad_token_id` to `eos_token_id`:13 for open-end generation.




> Entering new SequentialChain chain...


Setting `pad_token_id` to `eos_token_id`:13 for open-end generation.




> Entering new LLMChain chain...
Prompt after formatting:

You are the moderator of an online forum, you are strict and will not tolerate any negative comments.
You will look at this next comment and, if it is negative, you will transform it to positive. Avoid any negative words.
If it is nice, you will let it remain as is and repeat it word for word.
###
Original comment: You are a terrible person who is a complete waste of time and space.
###
Edited comment:

> Finished chain.

> Finished chain.


' \nYou are a great person and a great contributor to open source software.'

Sometimes the Moderator works fine, sometimes not. The Model is not able to detect correctly the sentiment, and if it detects the sentimet the comment modified can be worse than the origina or have no sense at all.

The model used is far to be a state of art Model, is really far from what we can obtain with model offered by OpenAI, or OpenSource models like LLAMA.

If you want to check how this same solutions works with OpenAI API check this notebook: https://github.com/peremartra/Large-Language-Model-Notebooks-Course/blob/main/LangChain_OpenAI_Moderation_System.ipynb